**MAESTRÍA EN INTELIGENCIA ARTIFICIAL APLICADA**

**Curso: TC4029 - Ciencia y analítica de datos**

Tecnológico de Monterrey

Prof Grettel Barceló Alonso

**Semana 8**
Regresión Lineal

---

NOMBRE: Carlos Eduardo García García
MATRÍCULA: A01795206
NOMBRE: Francisco Marmolejo Martinez
MATRÍCULA: A01794882
NOMBRE: Hugo Jovan Ramirez Ceron
MATRÍCULA: A00994966
Manuel Enrique Rodríguez Rey
MATRÍCULA: A01795392

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

2024-06-02 01:08:11.673595: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-02 01:08:11.673642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-02 01:08:11.675004: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-02 01:08:12.763695: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
melate_df = pd.read_csv('Tris.csv')
melate_df

,NPRODUCTO,CONCURSO,R1,R2,R3,R4,R5,FECHA,Multiplicador
0,60,32457,7,9,3,2,3.0,31/05/2024,SI
1,60,32456,0,0,3,9,6.0,31/05/2024,NO
2,60,32455,8,3,4,7,6.0,31/05/2024,SI
3,60,32454,2,2,1,6,2.0,31/05/2024,NO
4,60,32453,2,1,1,0,3.0,31/05/2024,NO
...,...,...,...,...,...,...,...,...,...
29539,60,2917,3,6,9,9,NaN,22/07/1996,NaN
29540,60,2916,2,8,8,2,NaN,21/07/1996,NaN
29541,60,2915,5,2,8,0,NaN,21/07/1996,NaN
29542,60,2914,9,5,4,6,NaN,20/07/1996,NaN


# **Parte 1**. EDA

Efectúa una exploración inicial de los datos a través de:

1a) Estadísticas descriptivas para todas las variables del dataframe.

In [4]:
dfNumericalValues = melate_df.copy().select_dtypes(exclude=['object'])
dfNumericalValues.describe()

,NPRODUCTO,CONCURSO,R1,R2,R3,R4,R5
count,29544.0,29544.000000,29544.000000,29544.000000,29544.000000,29544.000000,21425.000000
mean,60.0,17685.278466,4.506803,4.484125,4.489846,4.497800,4.490922
std,0.0,8529.061226,2.874030,2.876584,2.872701,2.873189,2.870904
min,60.0,2913.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,60.0,10299.750000,2.000000,2.000000,2.000000,2.000000,2.000000
50%,60.0,17685.500000,5.000000,4.000000,5.000000,5.000000,5.000000
75%,60.0,25071.250000,7.000000,7.000000,7.000000,7.000000,7.000000
max,60.0,32457.000000,9.000000,9.000000,9.000000,9.000000,9.000000


1b) Búsqueda de valores faltantes.

In [5]:
melate_df.isna().sum()

NPRODUCTO            0
CONCURSO             0
R1                   0
R2                   0
R3                   0
R4                   0
R5                8119
FECHA                0
Multiplicador    26894
dtype: int64

1c) Histrogramas y boxplots para las variables cuantitativas y diagramas de barras con la frecuencia para las variables categóricas.

In [6]:
y = melate_df['CONCURSO'].copy()
melate_df = melate_df.drop('NPRODUCTO', axis=1)
melate_df = melate_df.drop('CONCURSO', axis=1)
melate_df = melate_df.drop('FECHA', axis=1)
#melate_df = melate_df.drop('BOLSA', axis=1)
melate_df = melate_df.drop('Multiplicador', axis=1)
melate_df = melate_df.drop('R5', axis=1)
x = melate_df

x


,R1,R2,R3,R4
0,7,9,3,2
1,0,0,3,9
2,8,3,4,7
3,2,2,1,6
4,2,1,1,0
...,...,...,...,...
29539,3,6,9,9
29540,2,8,8,2
29541,5,2,8,0
29542,9,5,4,6


In [7]:
x.shape

(29544, 4)

5. Divide el conjunto en entrenamiento y prueba (80:20) considerando:


*   El parámetro `random_state` con el valor de 1 para garantizar reproducibilidad.
*   Analiza los diagramas de barras obtenidos en el ejercicio 1c. Con los resultados observados, es conveniente usar la variable `smoker` en la estrategia de estratificación.
*   Como ya tienes separadas en `X` e `y` los predictores y salida, consulta cómo ejecutar la función `train_test_split()` para obtener `Xtrain, Xtest, ytrain, ytest` en una única instrucción.

In [8]:
x

,R1,R2,R3,R4
0,7,9,3,2
1,0,0,3,9
2,8,3,4,7
3,2,2,1,6
4,2,1,1,0
...,...,...,...,...
29539,3,6,9,9
29540,2,8,8,2
29541,5,2,8,0
29542,9,5,4,6


In [9]:
x.shape

(29544, 4)

In [10]:
x[:10]

,R1,R2,R3,R4
0,7,9,3,2
1,0,0,3,9
2,8,3,4,7
3,2,2,1,6
4,2,1,1,0
5,9,7,4,6
6,0,4,6,5
7,3,5,1,6
8,6,8,9,1
9,3,3,4,2


In [11]:
x = np.array(x)

In [12]:
print(x[1])

[0 0 3 9]


In [20]:
from PIL import Image
import numpy as np

# Supongamos que tienes una matriz llamada 'data' de dimensiones (29544, 4)
data = x # Ejemplo de matriz aleatoria

# Normalizar los datos para que estén en el rango de 0 a 255 (escala de grises)
normalized_data = (data / 10.0) * 255
normalized_data = normalized_data.astype(np.uint8)  # Convertir los datos a tipo de datos de imagen (uint8)

# Redimensionar la matriz en forma de imagen (n, m) donde 'n' es el número de píxeles y 'm' es el número de canales de color
n_pixels = int(np.sqrt(normalized_data.shape[0]))  # Calcular el tamaño del lado del cuadrado
image_data = normalized_data[:n_pixels**2].reshape((n_pixels, n_pixels))  # Redimensionar a un cuadrado

# Crear una imagen en escala de grises a partir de los datos
image = Image.fromarray(image_data, mode='L')  # 'L' indica escala de grises

# Guardar la imagen
image.save('output_image_square.png')

ValueError: cannot reshape array of size 118176 into shape (171,171)